In [1]:
#!pip install git+https://github.com/huggingface/transformers.git
#!pip install transformers datasets
#!pip install wandb
import warnings
import sqlite3
import pandas as pd
import json
from tqdm import tqdm
import random
import math
# import wandb
# from wandb.keras import WandbCallback

In [2]:
val_table=pd.read_csv("/gpfsdswork/projects/rech/zpf/uyf36me/finetuning_NLP/resources/perfect_pdfs.csv")
#test_path="/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/test_1.csv"
#test_table=pd.read_csv(test_path)

In [3]:
import ast

def fonts_in_single_para(val):
    literal=ast.literal_eval(val)
    combined_fonts=[font_line.strip() for font_line in literal]
    combined_para_fonts=" ".join(combined_fonts)
    combined_para_fonts=combined_para_fonts.split()
    return combined_para_fonts

def continuous_fonts_clubbed(val):
    """clubs fonts that are together in a group sequence"""
    fonts_unique_order=[]
    for font in val:
        try:
            last_font=fonts_unique_order[-1]
        except:
            fonts_unique_order.append(font)
            continue
        if(font!=last_font):
            fonts_unique_order.append(font)
            
    return fonts_unique_order


val_table["global_fonts"]=val_table["global_fonts"].apply(fonts_in_single_para)
val_table["global_fonts"]=val_table["global_fonts"].apply(continuous_fonts_clubbed)
val_table

,page_no,top_left,bot_right,grobid_text,pdf_alto_text,fonts,label,global_fonts,Normal,Superscipt,...,is_Serif,font_color_red,font_color_green,font_color_blue,is_bold_manual,is_italic_manual,is_serif_manual,is_math_manual,new_font_size,pdf_path
0,1,"(200.0, 1009.5555555555555)","(1239.388888888889, 1307.111111111111)",Gabidulin codes [1] can be seen as the analogs...,['Gabidulin[~end_of_font~] codes[~end_of_font~...,['font8 font8 font8 font8 font8 font8 fo...,basic,"[cmr10, cmti10, cmr10]",1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.006944,0.000000,0.000000,0.272725,1205.0345/Wachter-Zeh_BoundsGabidulin_ACCT2012...
1,1,"(200.0, 1314.4166666666665)","(1239.388888888889, 1544.2222222222224)","For Gabidulin codes, there is no polynomial-ti...",['For[~end_of_font~] Gabidulin[~end_of_font~] ...,['font8 font8 font8 font8 font8 font8 fo...,basic,"[cmr10, cmmi10, cmr10]",1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.008929,0.000000,0.000000,0.272725,1205.0345/Wachter-Zeh_BoundsGabidulin_ACCT2012...
2,1,"(200.0, 1551.5277777777776)","(1239.388888888889, 1781.3333333333335)","In this paper, we provide a lower and an upper...","['In[~end_of_font~] this[~end_of_font~] paper,...",['font8 font8 font8 font8 font8 font8 fo...,basic,[cmr10],1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.272725,1205.0345/Wachter-Zeh_BoundsGabidulin_ACCT2012...
3,2,"(200.0, 492.44444444444446)","(1239.388888888889, 638.8888888888889)","Let q be a power of a prime, let F q denote th...",['Let[~end_of_font~] q[~end_of_font~] be[~end_...,['font8 font10 font8 font8 font8 font8 f...,basic,"[cmr10, cmmi10, cmr10, msbm10, cmmi8, cmr10, c...",0.792500,0.051528,...,0.0,0.0,0.0,0.0,0.000000,0.414306,0.000000,0.052639,0.245332,1205.0345/Wachter-Zeh_BoundsGabidulin_ACCT2012...
4,2,"(200.0, 699.8055555555555)","(1239.361111111111, 827.9999999999999)",. The (usual) addition and the non-commutative...,['following[~end_of_font~] holds:[~end_of_font...,['font8 font8 font10 font8 font21 font10 ...,basic,"[cmr10, cmmi10, cmr10, cmr8, cmmi10, cmr10, cm...",0.940769,0.019231,...,0.0,0.0,0.0,0.0,0.000000,0.201795,0.000000,0.000000,0.268373,1205.0345/Wachter-Zeh_BoundsGabidulin_ACCT2012...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529298,31,"(225.5123947309031, 548.15097534589)","(1428.3099929919588, 688.0228331059878)","Example 7.20.Consider the TRS R of f(s(x), 0) ...",['Example[~end_of_font~] 7.20.[~end_of_font~] ...,['font28 font28 font4 font4 font4 font13 ...,uri:extthm.example.25,"[cmbx10, cmr10, cmsy10, cmr10, cmss10, cmsy10,...",0.928480,0.034483,...,0.0,0.0,0.0,0.0,0.027778,0.084770,0.027778,0.221871,0.267470,1102.3129/arxiv.pdf
529299,31,"(225.5123947309031, 699.0814344942514)","(1428.3099929919588, 763.3213500964258)","Here, however, we must not conclude linear run...","['Here,[~end_of_font~] however,[~end_of_font~]...",['font4 font4 font4 font4 font4 font4 fo...,uri:extthm.example.25,"[cmr10, cmsy10, cmr10]",1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.071429,0.272725,1102.3129/arxiv.pdf
529300,31,"(225.5123947309031, 899.5533164448088)","(1428.3377757189182, 1227.6714666810017)",All described techniques have been incorporate...,['All[~end_of_font~] described[~end_of_font~] ...,['font4 font4 font4 font4 font4 font4 fo...,basic,"[cmr10, cmti10, cmss10, cmr10, cmr8, cmr10, cm...",0.985119,0.014881,...,0.0,0.0,0.0,0.0,0.000000,0.047685,0.007937,0.000000,0.271632,1102.3129/arxiv.pdf
529301,31,"(225.5123947309031, 1238.7300680692654)","(1428.3377757189182, 1566.4870077073494)",Table 1 summarises the experimental results of...,['Table[~end_of_font~] 1[~end_of_font~] summar...,['font4 font4 font4 font4 font4 font4 fo...,basic,"[cmr10, cmsy10, cmmi10, cmr10, cmss10, cmr10]",1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.027778,0.000000,0.006944,0.272725,1102.3129/arxiv.pdf


In [4]:
val_table["global_fonts"][100]

['msbm10',
 'cmmi6',
 'cmr6',
 'cmr8',
 'cmmi6',
 'cmr6',
 'cmr8',
 'cmr10',
 'cmss10',
 'cmmi8',
 'cmr10',
 'cmsy10',
 'cmr10',
 'cmss10',
 'cmr10',
 'msbm10',
 'cmr10',
 'cmss10',
 'cmr10']

In [5]:
test=pd.DataFrame(val_table["global_fonts"].apply(len))
test.describe()

,global_fonts
count,529303.000000
mean,23.795603
std,36.860435
min,0.000000
25%,5.000000
50%,14.000000
75%,29.000000
max,1528.000000


In [6]:
#paragraphs that use more than thresh set unique/not unique types of different fonts
df=val_table
thresh=1000
print(len(df["global_fonts"][df["global_fonts"].apply(len)>thresh]))
more_number_of_font=df[df["global_fonts"].apply(len)>thresh][["global_fonts","pdf_path","label"]]
more_number_of_font

10


,global_fonts,pdf_path,label
7181,"[nimbusromno9l, cmr10, cmsy10, cmmi10, cmmi7, ...",1806.00661/PIRCSI_Anoosheh_V4_longer_version.pdf,basic
98151,"[cmr10, cmmi7, cmsy10, cmmi7, cmr10, nimbusrom...",1207.0334/gap.pdf,basic
146180,"[nimbusromno9l, cmr10, nimbusromno9l, cmr10, n...",1802.09099/Pareto_mp_arxiv.pdf,basic
146181,"[cmsy10, cmmi7, cmr10, cmsy10, cmmi7, cmr10, c...",1802.09099/Pareto_mp_arxiv.pdf,basic
146182,"[cmr10, cmmi7, cmr10, cmmi10, cmsy7, cmmi7, cm...",1802.09099/Pareto_mp_arxiv.pdf,basic
146183,"[nimbusromno9l, cmmi10, cmsy10, nimbusromno9l,...",1802.09099/Pareto_mp_arxiv.pdf,basic
253967,"[nimbusromno9l, cmmi10, cmsy10, nimbusromno9l,...",1907.01133/ms.pdf,basic
268746,"[nimbusromno9l, cmmi10, nimbusromno9l, cmsy10,...",1806.02127/ms.pdf,uri:extthm.proof.4
316781,"[nimbusromno9l, cmsy10, cmmi7, cmmi10, cmsy10,...",1701.03559/ThR_DPM_arXiv.pdf,basic
412946,"[cmr10, cmr8, cmex7, cmbx5, cmsy5, cmmi5, cmr5...",1503.04661/FoundationsCollatz_Arxiv_ProofRead.pdf,basic


# train test split(As per NLP)

In [7]:
pdfs=list(df["pdf_path"].unique())

In [8]:
print(len(pdfs))

3682


## Problem1: No font sequences for some blocks

In [9]:
count=0
for i,element in enumerate(df["global_fonts"]):
    if(len(element)==0):
        print(df.iloc[i])
        count+=1
        break
print("total places in Total data that have no sequence information:{}".format(count))

page_no                                                   4
top_left            (866.6111111111111, 1885.8888888888887)
bot_right           (934.2222222222222, 1909.8888888888887)
grobid_text                                           where
pdf_alto_text                                          ['']
fonts                                                  ['']
label                                      uri:extthm.thm.2
global_fonts                                             []
Normal                                                  0.0
Superscipt                                              0.0
Subscript                                               0.0
italics                                                 0.0
bold                                                    0.0
is_Proportional                                         0.0
is_Serif                                                0.0
font_color_red                                          0.0
font_color_green                        

In [10]:
count=0
for i,element in enumerate(val_table["global_fonts"]):
    if(len(element)==0):
        print("yo")
        print(val_table.iloc[i])
        count+=1
        break
print("total places in val data that have no sequence information:{}".format(count))

yo
page_no                                                   4
top_left            (866.6111111111111, 1885.8888888888887)
bot_right           (934.2222222222222, 1909.8888888888887)
grobid_text                                           where
pdf_alto_text                                          ['']
fonts                                                  ['']
label                                      uri:extthm.thm.2
global_fonts                                             []
Normal                                                  0.0
Superscipt                                              0.0
Subscript                                               0.0
italics                                                 0.0
bold                                                    0.0
is_Proportional                                         0.0
is_Serif                                                0.0
font_color_red                                          0.0
font_color_green                     

## Problem2: More than 500 font sequences for some blocks

In [11]:
count=0
thresh=500
for i,element in enumerate(df["global_fonts"]):
    if(len(element)>=thresh):
        print("yo")
        print(df.iloc[i])
        count+=1
        break
print("total places in data that have no sequence information:{}".format(count))

yo
page_no                                                             3
top_left                                  (136.0, 154.94444444444443)
bot_right                                (1564.0000000000002, 2084.0)
grobid_text         2 , D (j) ).It is easy to see that each of the...
pdf_alto_text       ['Proof.[~end_of_font~] The[~end_of_font~] pro...
fonts               ['font10  font10  font10  font10  font10  font...
label                                                           basic
global_fonts        [nimbusromno9l, cmmi9, cmr9, cmsy9, nimbusromn...
Normal                                                       0.832692
Superscipt                                                   0.067653
Subscript                                                    0.099654
italics                                                           0.0
bold                                                              0.0
is_Proportional                                              0.989936
is_Serif         

# building vocabulary ignore if already built

#we need to build the training vocabulary
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import glob
from tqdm import tqdm

top_words = 52000 


tokenizer = Tokenizer(num_words=top_words,lower=False,oov_token="UNK")

#traverse through all the training file csvs
#df["global_fonts"]=df["global_fonts"].apply(fonts_in_single_para)
#df["global_fonts"]=df["global_fonts"].apply(continuous_fonts_clubbed)

train_path="/Volumes/My_Book/Theoremkb/cluster_package/8-finetuning_data/train_data/**.csv"

files=glob.glob(train_path)

for i,file in tqdm(enumerate(files)):
    df=pd.read_csv(file)
    df["global_fonts"]=df["global_fonts"].apply(fonts_in_single_para)
    df["global_fonts"]=df["global_fonts"].apply(continuous_fonts_clubbed)
    tokenizer.fit_on_texts(df["global_fonts"])
    
    

#tokenizer.fit_on_texts(val_table["global_fonts"])
#list_tokenized_train = tokenizer.texts_to_sequences(val_table["global_fonts"])

#for bert max length is 512 
#X_train = pad_sequences(list_tokenized_train, maxlen=max_length)
#X_train

#save the tokenizer to be able to load it further
print(len(tokenizer.word_counts))
import pickle

#fitting on validation data
#val_table["global_fonts"]=val_table["global_fonts"].apply(fonts_in_single_para)
#val_table["global_fonts"]=val_table["global_fonts"].apply(continuous_fonts_clubbed)
tokenizer.fit_on_texts(val_table["global_fonts"])
print(len(tokenizer.word_counts))


#fitting on test data
test_table["global_fonts"]=test_table["global_fonts"].apply(fonts_in_single_para)
test_table["global_fonts"]=test_table["global_fonts"].apply(continuous_fonts_clubbed)
tokenizer.fit_on_texts(test_table["global_fonts"])
print(len(tokenizer.word_counts))


#saving the model
#filepath = "tokenizer_52000.pkl"
#with open(filepath, 'wb') as f:
    #pickle.dump(tokenizer, f)

In [12]:
#filepath = "tokenizer_52000_v1.pkl"
#with open(filepath, 'wb') as f:
    #pickle.dump(tokenizer, f)

In [13]:
#load tokenizer construct validation dataset

#read the tokenizer
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import glob
from tqdm import tqdm


filepath = "tokenizer_52000_v1.pkl"

with open(filepath, 'rb') as f:
    tokenizer = pickle.load(f)
    print(len(tokenizer.word_counts))
    
max_length=1000

val_tokenized_train = tokenizer.texts_to_sequences(val_table["global_fonts"])
X_val = pad_sequences(val_tokenized_train, maxlen=max_length)

#test_tokenized_train = tokenizer.texts_to_sequences(test_table["global_fonts"])
#X_test = pad_sequences(test_tokenized_train, maxlen=max_length)


2023-04-20 12:26:10.290425: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


4031


In [14]:
print(X_val.shape)
#print(X_test.shape)

(529303, 1000)


In [15]:
print(len(X_val))
print(X_val[0])
sample=X_val[15]

529303
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  

In [16]:
tokenizer.sequences_to_texts([sample]) #if giving a single value then that has to be passed through the list

['UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UN

In [17]:
#freeze the base model
def setting_up_labels(label):
    label=label.lower()
    if("basic" in label):
        return "basic"
    if("overlap" in label):
        return "overlap"
    if("proof" in label):
        return "proof"
    if("theorem" in label):
        return  "theorem"
    else:
        return "theorem"
    
    
def setting_up_labels_1(label):
    label=label.lower()
    if("basic" in label):
        return 0
    if("overlap" in label):
        return 3
    if("proof" in label):
        return 1
    if("theorem" in label):
        return  2
    
    



In [18]:
#similarly we can get y_train and y_test
#y_test=test_table["label"].to_numpy()
val_table["label"]=val_table["label"].apply(setting_up_labels)
val_table=val_table.dropna()
val_table["label"]=val_table["label"].apply(setting_up_labels_1)
y_val=val_table["label"].to_numpy()

In [19]:
print(X_val.shape,y_val.shape)

(529303, 1000) (529303,)


## convert into dataset objects

In [20]:
import tensorflow as tf

strategy = tf.distribute.MirroredStrategy()

print("no of devices: {}".format(strategy.num_replicas_in_sync))
devices = strategy.num_replicas_in_sync

2023-04-20 12:27:23.201606: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-20 12:27:25.650025: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38155 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:14:00.0, compute capability: 8.0
2023-04-20 12:27:25.674942: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38280 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:15:00.0, compute capability: 8.0
2023-04-20 12:27:25.676663: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created 

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
no of devices: 4


In [21]:
#test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).shuffle(buffer_size=1024)

In [31]:
batch_size=16 #originally 64

batch_size=batch_size*devices
#test_dataset=test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE).batch(batch_size)
val_dataset=val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE).batch(batch_size)

## Running LSTM cell

In [32]:
train_path="/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/**.csv"
import glob
files=glob.glob(train_path)
files

['/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_28.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_18.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_108.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_106.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_85.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_123.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_7.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_48.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_2.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_114.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_125.csv',
 '/link

In [33]:
#convert everything to tf dataset object
#GRU's work worse than LSTM

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import  Dense, LSTM,GRU, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow_addons.optimizers import AdamW ,LAMB
# This is fixed.


EMBEDDING_DIM = 32 ####hyper parameter
learning_rate=0.001
top_words = 52000 

#opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
#opt=AdamW(learning_rate)
opt=LAMB(learning_rate)


with strategy.scope():
    model = Sequential()

    model.add(Embedding(top_words, EMBEDDING_DIM))
    #model.add(SpatialDropout1D(0.1))

    model.add(LSTM(128)) #for cudadnn set rec dropout as 0 Bidirectional()
    #model.add(LSTM(64)) #for cudadnn set rec dropout as 0
    #model.add(LSTM(100, dropout=0.2)) #for cudadnn set rec dropout as 0

    model.add(Dense(4, activation='softmax'))
    model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          1664000   
                                                                 
 lstm_1 (LSTM)               (None, 128)               82432     
                                                                 
 dense_1 (Dense)             (None, 4)                 516       
                                                                 
Total params: 1,746,948
Trainable params: 1,746,948
Non-trainable params: 0
_________________________________________________________________


In [34]:
val_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, None, 1000), dtype=tf.int32, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>

In [35]:
#reordering file name accroding to some predefined run (which is here nlp)

file_path="/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/roberta_from_scratch_ft.txt"

import os
base="/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/"
with open(file_path,"r") as f:
    lines=f.readlines()
    
start=[]
for line in lines:
    val=line.split(",")[0]
    start.append(val)

for file in files:
    if(file in start):
        continue
    else:
        start.append(file)
        
files=start
files

['/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_28.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_18.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_7.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_2.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_0.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_108.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_106.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_85.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_123.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_48.csv',
 '/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_114.csv',
 '/linkho

In [36]:
def get_last_run(log_file):
    with open(log_file,"r") as f:
        lines=f.readlines()
    f.close()
    
    files=[]
    for line in lines:
        line=line.replace("./training_data/","/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/")
        files.append(line.split(",")[0])
        
    return files

def append_last_run(val,log_file):
    with open(log_file,"a") as f:
        f.write(val)


log_file="./font_clf_blstm_res.txt"
#get_last_run(log_file)

In [37]:
log_file="./bgru_font.txt"
model_path="./bgru_font.h5"


if(os.path.exists(model_path)):
    with strategy.scope():
        model = tf.keras.models.load_model(model_path)

#skips=["/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_93.csv"]

for i,file in tqdm(enumerate(files)):
    #get readings from past run
    try:
        run_so_far=get_last_run(log_file)
        if(file in run_so_far ): #or file in skips
            continue
        else:
            print(file)
            print("not in previous runs, hence running on this batch")
    except FileNotFoundError:
        pass
    print("doing--", file)
    df=pd.read_csv(file)
    df["global_fonts"]=df["global_fonts"].apply(fonts_in_single_para)
    df["global_fonts"]=df["global_fonts"].apply(continuous_fonts_clubbed)
    df["label"]=df["label"].apply(setting_up_labels)
    df=df.dropna()
    df["label"]=df["label"].apply(setting_up_labels_1)
    df_tokenized_train = tokenizer.texts_to_sequences(df["global_fonts"])
    df_x = pad_sequences(df_tokenized_train, maxlen=max_length)
    df_y=df["label"].to_numpy()
    
    
    df_dataset = tf.data.Dataset.from_tensor_slices((df_x, df_y)).shuffle(buffer_size=1024)
    df_dataset=df_dataset.prefetch(buffer_size=tf.data.AUTOTUNE).batch(batch_size)
    
    with strategy.scope():
        history = model.fit(df_dataset,validation_data=val_dataset,epochs=1)
        #model_res= model.evaluate(val_dataset)
        #val_loss,val_acc=model_res[0],model_res[1]
        #metrics after a batch
        train_loss=history.history['loss']
        val_loss=history.history['val_loss']
        train_acc=history.history['sparse_categorical_accuracy']
        val_acc=history.history['val_sparse_categorical_accuracy']
        
        line_to_write=f'{file},{train_loss},{val_loss},{train_acc},{val_acc}\n'
        append_last_run(line_to_write,log_file)
        
        #saving the model
        model.save(model_path)
        

0it [00:00, ?it/s]

/linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_2.csv
not in previous runs, hence running on this batch
doing-- /linkhome/rech/gennsp01/uyf36me/work/finetuning_NLP/resources/train_data/batch_2.csv


2023-04-20 14:43:14.992707: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT32
      type: DT_INT64
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 187398
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\026TensorSliceDataset:126"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 1000
        }
      }
      shape {
      }
    }
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TF

INFO:tensorflow:batch_all_reduce: 8 all-reduces with algorithm = nccl, num_packs = 1
2928/2929 [============================>.] - ETA: 0s - loss: 0.7071 - sparse_categorical_accuracy: 0.7269

2023-04-20 14:52:01.825899: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at cudnn_rnn_ops.cc:1563 : UNKNOWN: CUDNN_STATUS_BAD_PARAM
in tensorflow/stream_executor/cuda/cuda_dnn.cc(1593): 'cudnnSetTensorNdDescriptor( tensor_desc.get(), data_type, sizeof(dims) / sizeof(dims[0]), dims, strides)'
2023-04-20 14:52:01.826449: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at cudnn_rnn_ops.cc:1563 : UNKNOWN: CUDNN_STATUS_BAD_PARAM
in tensorflow/stream_executor/cuda/cuda_dnn.cc(1593): 'cudnnSetTensorNdDescriptor( tensor_desc.get(), data_type, sizeof(dims) / sizeof(dims[0]), dims, strides)'
3it [09:08, 182.72s/it]


CancelledError: Graph execution error:

RecvAsync is cancelled.
	 [[{{node cond/output/_15/_138}}]] [Op:__inference_train_function_172513]

In [ ]:
#test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
batch_size=64
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset=val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE).batch(batch_size)

from sklearn.metrics import classification_report, confusion_matrix, f1_score
from tensorflow_addons.optimizers import AdamW ,LAMB
import numpy as np


labels=None
for x, y in tqdm(val_dataset):
    if(labels is None):
        labels=y
    else:
        labels=np.concatenate([labels,y])

In [ ]:
#ground truth        
y_true=labels

def evaluate_f1_for_tf_model(model_path,validation_dataset,y_true,show_confusion_report=True):
    
    # 4 A100 can do the job in

    class_names=["Basic","Proof","Theorem","Overlap"]

    # Wrap the loaded model inside the strategy scope to distribute it across the GPUs
    with strategy.scope():
        model = tf.keras.models.load_model(model_path)

    #show model arch
    print(model.summary())
    
    
    #generating predictions
    predictions=model.predict(validation_dataset)
    
    #generating predictions
    y_pred = np.argmax(predictions, axis=1)
    
    if(show_confusion_report is True):
        print('Confusion Matrix')
        print(classification_report(y_true, y_pred, target_names=class_names))
        
    return f1_score(y_true,y_pred,average="macro")

model="./my_model_blstm_128.h5"
_f1_score=evaluate_f1_for_tf_model(model_path=model,validation_dataset=val_dataset,y_true=y_true)
print(f"f1 score of the {model} is {_f1_score}") 


In [ ]:
labels.shape

In [ ]:
df_dataset

In [ ]:
model.save("./my_model_lstm_100.h5")

bi directional part fails

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 embedding_7 (Embedding)     (None, 1000, 32)          16000     
                                                                 
 spatial_dropout1d_7 (Spatia  (None, 1000, 32)         0         
 lDropout1D)                                                     
                                                                 
 lstm_6 (LSTM)               (None, 100)               53200     
                                                                 
 dense_7 (Dense)             (None, 3)                 303       
                                                                 
=================================================================
Total params: 69,503
Trainable params: 69,503
Non-trainable params: 0

#simple lstm

19500/19500 [==============================] - ETA: 0s - loss: 0.7095 - accuracy: 0.7013
2022-04-11 12:39:51.713498: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.

2022-04-11 12:39:51.772903: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.

19500/19500 [==============================] - 5009s 256ms/step - loss: 0.7095 - accuracy: 0.7013 - val_loss: 0.6975 - val_accuracy: 0.7053

Epoch 2/30
19500/19500 [==============================] - 4937s 253ms/step - loss: 0.6849 - accuracy: 0.7105 - val_loss: 0.6912 - val_accuracy: 0.7085

Epoch 3/30
19500/19500 [==============================] - 5082s 261ms/step - loss: 0.6788 - accuracy: 0.7123 - val_loss: 0.6884 - val_accuracy: 0.7100

Epoch 4/30
10236/19500 [==============>...............] - ETA: 37:42 - loss: 0.6776 - accuracy: 0.7125

In [ ]:
 #embedding dimension
#LSTM cells number
#recurrent drop outs
#dropouts
#spatial dropouts

#LSTM 600 of 32 gives acc or loss
#bi directional improves


In [ ]:
#bayesian optimization

import optuna


def objective(trial):
    
    #some initial things
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    batch_size=32
    model = Sequential()
    
    # Categorical parameter
    EMBEDDING_DIM=trial.suggest_int("num_units", 16, 768,log=True)
    model.add(Embedding(top_words, EMBEDDING_DIM))
    
    #spatial dropout
    spatial_dropout_rate = trial.suggest_float("spatial_dropout_rate", 0.0, 1.0)
    model.add(SpatialDropout1D(0.2))
    
    ## Floating point parameter dropout same as dropout rate
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 1.0)
    
    # Integer parameter
    n_layers = trial.suggest_int('n_layers', 1, 5)

    for i in range(n_layers):
        if(i!=n_layers-1):
            num_hidden = trial.suggest_int(f'lstm_units_l{i}', 4, 512, log=True)
            model.add(LSTM(num_hidden, dropout=dropout_rate,return_sequences=True)) #for cudadnn set rec dropout as 0
        else:
            model.add(LSTM(num_hidden, dropout=dropout_rate))
        
    model.add(Dense(3, activation='softmax'))
    model.compile(loss="sparse_categorical_crossentropy", optimizer=opt, metrics=['accuracy'])
    model.summary()
    
    epochs = 3 #hyper parameter not really that important

    batch_size = 32 #number of training samples to push through
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                      validation_data=(X_val,y_val),
                      callbacks=[EarlyStopping(monitor='val_loss',mode="min",  patience=3, min_delta=0.01)]
                      )
    
    return hist.history['val_loss'][-1]
    
# Creating a study object and optimizing the objective function.
study = optuna.create_study(direction='minimize',study_name="NN_tuning",storage="sqlite:///nn_net.db",load_if_exists=True)
# n_trials means number of trials
study.optimize(objective, n_trials=100)    

In [ ]:

# Again loading the object from the nn_net file and updating it again.
study = optuna.create_study(direction='minimize',
                            sampler=optuna.samplers.TPESampler(seed=5),
                            study_name="NN_tuning",
                            storage="sqlite:///nn_net.db",
                            load_if_exists=True)
import pandas as pd
print("best_params",study.best_params)
print(study.best_value)
print("best trial--",study.best_trial)

In [ ]:
study=optuna.study.load_study(storage="sqlite:///nn_net.db",study_name="NN_tuning")
import pandas as pd
print("best_params",study.best_params)
print(study.best_value)
print("best trial--",study.best_trial)

In [ ]:
from optuna.visualization import plot_optimization_history
plot_optimization_history(study)


In [ ]:
from optuna.visualization import plot_parallel_coordinate
plot_parallel_coordinate(study)

In [ ]:
from optuna.visualization import plot_param_importances
plot_param_importances(study)